<a href="https://colab.research.google.com/github/itzdineshx/Internship_projects--DINESH-S-/blob/main/Twitter_sentimental_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TWITTER SENTIMENTAL ANALYSIS

In [ ]:
 #installing kaggle for data
!pip install kaggle

Uploading kaggle dataset by API


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Importing dataset by the above API


In [ ]:
#fetching data through api
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 91% 74.0M/80.9M [00:00<00:00, 105MB/s] 
100% 80.9M/80.9M [00:00<00:00, 90.2MB/s]


In [ ]:
#extracting the data
from zipfile import ZipFile
file_name='/content/sentiment140.zip'
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('done')#when printed done the extracion is done

done


# **About Dataset**
Context
This is the sentiment140 dataset
It contains 1,600,000 tweets extracted using the twitter api .
Content
It contains the following 6 fields:
target: the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)
ids: The id of the tweet

1. date: the date of the tweet
2. flag: The query
3. user: the user that tweeted
4. text: the text of the tweet

data set link : [twitter data](https://www.kaggle.com/datasets/kazanova/sentiment140)


importing the all wanted libraries


In [ ]:
import pandas as pd #for pre-processing data
import numpy as np #for numerical calculations
import re #for search pattern recognition
from nltk.corpus import stopwords #for stopwords
from nltk.stem import PorterStemmer #for stemming
from sklearn.feature_extraction.text import TfidfVectorizer #for vectorization
from sklearn.model_selection import train_test_split #for splitting train and test the data
from sklearn.linear_model import LogisticRegression #for logistic regression
from sklearn.metrics import accuracy_score #for accuracy

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# **stopwords:**
Stopwords are common words in a language that are often filtered out during NLP tasks because they are considered to carry little meaning or contribute minimally to the overall understanding of a text.

In [ ]:
#printing all the stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# **Data Processing**

In [ ]:
#loadin the data
twit_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

In [ ]:
#checking the no of rows and columns
twit_data.shape

(1599999, 6)

In [ ]:
#checking the first and last 5 rows
twit_data.head()
twit_data.tail()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...
1599998,4,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...


In [ ]:
#checking the information of the file
twit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 6 columns):
 #   Column                                                                                                               Non-Null Count    Dtype 
---  ------                                                                                                               --------------    ----- 
 0   0                                                                                                                    1599999 non-null  int64 
 1   1467810369                                                                                                           1599999 non-null  int64 
 2   Mon Apr 06 22:19:45 PDT 2009                                                                                         1599999 non-null  object
 3   NO_QUERY                                                                                                             1599999 non-null  object
 4   _

In [ ]:
#naming the coumns
column_names=['target','ids','date','flag','user','text']
twit_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_names,encoding='ISO-8859-1')

In [ ]:
#checking the no of rows and columns
twit_data.shape


(1600000, 6)

In [ ]:
#checking the first and last 5 rows
twit_data.head()
twit_data.tail()

,target,ids,date,flag,user,text
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...
1599999,4,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...


In [ ]:
#counting the null values
twit_data.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [ ]:
#checking the distribution
twit_data['target'].value_counts()

,count
target,
0,800000
4,800000


by analyzing the above distribution there is no neutral values as 2 and also the data is evenly distributed as 800k each.so no up levelling and low levelling the data to train the model

converting target values 4's to 1's

In [ ]:
twit_data.replace({'target':{4:1}},inplace=True)

In [ ]:
#checking the distribution again
twit_data['target'].value_counts()

,count
target,
0,800000
1,800000




*  0 --> means negative tweet
* 1 --> means positive **tweet**



# **stemming**:
Stemming is a text normalization technique in natural language processing that reduces words to their base or root form, removing prefixes or suffixes.
ex: player,playing,plays --> plays

In [ ]:
port_stem = PorterStemmer()
def stemming(content):
    stemmed = re.sub('[^a-zA-Z]',' ',content)
    stemmed = stemmed.lower()#converting to lowercase
    stemmed = stemmed.split()#spliting data
    stemmmed = [port_stem.stem(word) for word in stemmed if not word in stopwords.words('english')]#removing stopwords
    stemmed = ' '.join(stemmed)
    return stemmed

In [ ]:
twit_data['text'] = twit_data['text'].apply(stemming)

In [ ]:
twit_data.head()


,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,switchfoot http twitpic com y zl awww that s a...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can t update his facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,kenichan i dived many times for the ball manag...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,nationwideclass no it s not behaving at all i ...


In [ ]:
print(twit_data['text'])

0          switchfoot http twitpic com y zl awww that s a...
1          is upset that he can t update his facebook by ...
2          kenichan i dived many times for the ball manag...
3             my whole body feels itchy and like its on fire
4          nationwideclass no it s not behaving at all i ...
                                 ...                        
1599995    just woke up having no school is the best feel...
1599996    thewdb com very cool to hear old walt intervie...
1599997    are you ready for your mojo makeover ask me fo...
1599998    happy th birthday to my boo of alll time tupac...
1599999    happy charitytuesday thenspcc sparkscharity sp...
Name: text, Length: 1600000, dtype: object


In [ ]:
print(twit_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
x = twit_data['text'].values
y = twit_data['target'].values
print(x)
print(y)

['switchfoot http twitpic com y zl awww that s a bummer you shoulda got david carr of third day to do it d'
 'is upset that he can t update his facebook by texting it and might cry as a result school today also blah'
 'kenichan i dived many times for the ball managed to save the rest go out of bounds'
 ... 'are you ready for your mojo makeover ask me for details'
 'happy th birthday to my boo of alll time tupac amaru shakur'
 'happy charitytuesday thenspcc sparkscharity speakinguph h']
[0 0 0 ... 1 1 1]


# **Splitting the data**


*   Train Data
*   Test Data



In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)

converting the textual data -> Numerical data by vectorization

In [ ]:
vectorizer = TfidfVectorizer()

x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)


In [ ]:
print(x_train)
print(x_test)

  (0, 488987)	0.41322347757019917
  (0, 258884)	0.3855833943595694
  (0, 121501)	0.38436070312484305
  (0, 17690)	0.1496211864872621
  (0, 206332)	0.4857480076242065
  (0, 390656)	0.3293975539873257
  (0, 481939)	0.30447551653830146
  (0, 453363)	0.12148485475980697
  (0, 2067)	0.24193877001313313
  (1, 199337)	0.21534180706932754
  (1, 178952)	0.9765387376484952
  (2, 316531)	0.1431133085476043
  (2, 147379)	0.176274482460971
  (2, 179184)	0.10574792677385002
  (2, 167881)	0.16030807228052635
  (2, 197952)	0.1380493162283387
  (2, 451067)	0.133366032887775
  (2, 12324)	0.12234713047008548
  (2, 293054)	0.20846695577578972
  (2, 489389)	0.3537352115264888
  (2, 442340)	0.10230075002928594
  (2, 85757)	0.2679936085012617
  (2, 478415)	0.281805932852511
  (2, 442727)	0.14215717985609486
  (2, 448054)	0.28650103709365976
  :	:
  (1279998, 138593)	0.26304278076663656
  (1279998, 485504)	0.20346683466366844
  (1279998, 188630)	0.22920711233364058
  (1279998, 175223)	0.21336006149095368
  (1

Training the ML model
Logistic Regression: To check the logical output of 0s and 1s

In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(x_train,y_train)

LogisticRegression(max_iter=1000)

Model Evaluation:
Accuracy Score

In [ ]:
#accuracy score
x_train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(x_train_prediction,y_train)
print(f"Accuracy Score={training_data_accuracy}")

Accuracy Score=0.82785


In [ ]:
#accuracy score
x_test_prediction = model.predict(x_test)
test_data_accuracy = accuracy_score(x_test_prediction,y_test)
print(f"Accuracy Score={test_data_accuracy}")

Accuracy Score=0.80215625


saving the trained model

In [ ]:
import pickle #for saving the model
filename = 'trained_model.sav'
pickle.dump(model,open(filename,'wb'))

In [ ]:
#loading the model
loaded_model = pickle.load(open('/content/trained_model.sav','rb'))

# **Executing Results:**

In [ ]:
indx = int(input("Enter the index number:"))
x_new = x_test[indx]
prediction = loaded_model.predict(x_new)
print(prediction)
if (prediction[0]==0):
  print('Negative Tweet')
else:
  print('Positive Tweet')

Enter the index number:0
[1]
Positive Tweet
